In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np


In [16]:
# Replace 'data.xlsx' with your Excel file path
excel_file = 'AirStability1.xlsx'

# Define column pairs by their positions (0-based indexing)
# Example: [(0,1), (2,3)] corresponds to columns 1-2 and 3-4
column_pairs = [(0, 1), (2, 3)]  # Adjust as needed for more pairs

# Read the required columns
usecols = [col for pair in column_pairs for col in pair]  # Flatten the list
df = pd.read_excel(excel_file, usecols=usecols)

# Drop rows with missing values in any of the selected columns
df = df.dropna(subset=df.columns)

# Sample the data to reduce the number of frames
sampling_rate = 10  # Adjust based on dataset size
df_sampled = df.iloc[::sampling_rate].reset_index(drop=True)

# Prepare data for each pair
data_series = []
for x_col, y_col in column_pairs:
    x = df_sampled.iloc[:, x_col]
    y = df_sampled.iloc[:, y_col]
    data_series.append({'x': x, 'y': y, 'label': df.columns[y_col]})

# Verify the prepared data
for series in data_series:
    print(series['label'], ':', len(series['x']), 'points')


BTY : 75 points
BTY UV : 75 points


In [20]:
# Define figure size in centimeters
fig_width_cm = 14
fig_height_cm = 9

# Convert centimeters to inches for Matplotlib
fig_width = fig_width_cm / 2.54
fig_height = fig_height_cm / 2.54

# Create a directory to save the figures
output_dir = 'organic_figures'
os.makedirs(output_dir, exist_ok=True)

# Function to create and save a figure for each organic
def create_figure(organic, organicUV, color_as_deposited, color_uv_treated, output_dir):
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    
    # Plot organic (As Deposited)
    ax.plot(
        organic['x'], 
        organic['y'], 
        color=color_as_deposited, 
        linestyle='solid', 
        linewidth=2,
        label='As Deposited'  # Label used only for the separate legend
    )
    
    # Plot organicUV (UV Treated)
    ax.plot(
        organicUV['x'], 
        organicUV['y'], 
        color=color_uv_treated, 
        linestyle='dashed', 
        linewidth=2,
        label='UV Treated'  # Label used only for the separate legend
    )
    
    # Set labels and title
    ax.set_xlabel('Time (minutes)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Normalized Measurement', fontsize=12, fontweight='bold')
    ax.set_title(f"{organic['label']} Measurements Over Time", fontsize=14, fontweight='bold')
    
    # Set Y-axis starting at 0 and Y-axis limits
    ax.set_ylim(bottom=y_min, top=y_max * 1.05)  # Slightly higher for aesthetics
    ax.set_xlim(x_min, x_max)
    
    # Add grid
    #ax.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    
    # Customize spines (all four spines visible for boxing)
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_linewidth(1)
        spine.set_color('black')
    
    # Set major and minor ticks
    ax.xaxis.set_major_locator(MultipleLocator(10))  # Major ticks every 10 minutes
    ax.xaxis.set_minor_locator(MultipleLocator(5))   # Minor ticks every 5 minutes
    ax.yaxis.set_major_locator(MultipleLocator(0.25))  # Major ticks every 0.25 units
    ax.yaxis.set_minor_locator(AutoMinorLocator(4))    # 3 minor ticks between major ticks
    
    # Customize tick parameters
    ax.tick_params(axis='both', which='major', labelsize=10, direction='in', length=6)
    ax.tick_params(axis='both', which='minor', labelsize=8, direction='in', length=3)
    
    # Exclude legend from individual figures
    # ax.legend(fontsize=10, loc='upper left')  # Commented out to exclude legend
    
    # Adjust layout for tightness
    plt.tight_layout()
    
    # Save the figure in high resolution
    figure_name = f"{organic['label']}.png"
    fig.savefig(os.path.join(output_dir, figure_name), dpi=300)
    
    # Close the figure to free memory
    plt.close(fig)

# Iterate through each organic and create figures
for org in organics:
    # Get the corresponding UV series
    org_uv_label = f"{org} UV"
    
    # Find the series dictionaries
    organic = next((s for s in organic_series if s['label'] == org), None)
    organicUV = next((s for s in organicUV_series if s['label'] == org_uv_label), None)
    
    if organic and organicUV:
        create_figure(
            organic=organic, 
            organicUV=organicUV, 
            color_as_deposited=color_as_deposited, 
            color_uv_treated=color_uv_treated, 
            output_dir=output_dir
        )
        print(f"Figure for {org} created successfully.")
    else:
        print(f"Series for {org} or {org_uv_label} not found.")


NameError: name 'os' is not defined

In [19]:
# Install required libraries
!pip install pandas matplotlib openpyxl

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np

# Define the list of organics
organics = ['MPD', 'EG', 'THB', 'BTY', 'DHB', 'CB']

# Generate the corresponding UV labels
organics_uv = [f"{organic} UV" for organic in organics]

# Combine into a single list for easier processing
all_series = organics + organics_uv

# Replace 'data.xlsx' with your actual Excel file path
excel_file = 'data.xlsx'

# Read the entire Excel file
df = pd.read_excel(excel_file)

# Identify column pairs (Assuming columns are in pairs: X1, Y1, X2, Y2, ...)
columns = df.columns.tolist()
column_pairs = []

for i in range(0, len(columns), 2):
    if i+1 < len(columns):
        column_pairs.append((columns[i], columns[i+1]))

# Initialize lists to hold organic and organicUV series
organic_series = []
organicUV_series = []

# Populate the lists based on Y column headers
for x_col, y_col in column_pairs:
    if y_col in organics:
        organic_series.append({'x': df[x_col], 'y': df[y_col], 'label': y_col})
    elif y_col in organics_uv:
        organicUV_series.append({'x': df[x_col], 'y': df[y_col], 'label': y_col})

# Verify the identified series
print("Organic Series:")
for series in organic_series:
    print(f" - {series['label']} with {len(series['x'])} data points")

print("\nOrganicUV Series:")
for series in organicUV_series:
    print(f" - {series['label']} with {len(series['x'])} data points")

# Define the time limit
time_limit = 60  # minutes

# Function to filter series up to the time limit
def filter_series(series):
    mask = series['x'] <= time_limit
    series['x'] = series['x'][mask]
    series['y'] = series['y'][mask]
    return series

# Apply filtering to all series
organic_series = [filter_series(s) for s in organic_series]
organicUV_series = [filter_series(s) for s in organicUV_series]

# Determine the overall X range
all_x = pd.concat([s['x'] for s in organic_series + organicUV_series])
x_min = all_x.min()
x_max = all_x.max()

# Set Y-axis minimum to 0 and determine Y range
all_y = pd.concat([s['y'] for s in organic_series + organicUV_series])
y_min = 0
y_max = all_y.max()

# Determine the maximum number of frames for organic and organicUV
max_frames_organic = max([len(s['x']) for s in organic_series], default=0)
max_frames_organicUV = max([len(s['x']) for s in organicUV_series], default=0)

# Total frames: animate organic first, then organicUV
total_frames = max_frames_organic + max_frames_organicUV

# Initialize the plot
fig, ax = plt.subplots(figsize=(12, 8))

# Initialize lines for organic series (solid lines)
lines_organic = []
for series in organic_series:
    line, = ax.plot([], [], lw=2, label=series['label'], color='blue')
    lines_organic.append(line)

# Initialize lines for organicUV series (dashed lines)
lines_organicUV = []
for series in organicUV_series:
    line, = ax.plot([], [], lw=2, label=series['label'], color='red', linestyle='--')
    lines_organicUV.append(line)

# Set plot limits
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)

# Set labels and title
ax.set_xlabel('Time (minutes)', fontsize=14)
ax.set_ylabel('Measurement', fontsize=14)
ax.set_title('Growing Line Plot Animation for Organics and OrganicUV', fontsize=16)
ax.grid(True)

# Create a legend
ax.legend(loc='upper left')

# Initialize data lists
xdata_organic = [[] for _ in organic_series]
ydata_organic = [[] for _ in organic_series]

xdata_organicUV = [[] for _ in organicUV_series]
ydata_organicUV = [[] for _ in organicUV_series]

# Initialization function
def init():
    for line in lines_organic + lines_organicUV:
        line.set_data([], [])
    return lines_organic + lines_organicUV

# Animation function
def animate(frame):
    # Phase 1: Animate organic lines
    if frame < max_frames_organic:
        for idx, series in enumerate(organic_series):
            if frame < len(series['x']):
                xdata_organic[idx].append(series['x'].iloc[frame])
                ydata_organic[idx].append(series['y'].iloc[frame])
                lines_organic[idx].set_data(xdata_organic[idx], ydata_organic[idx])
    # Phase 2: Animate organicUV lines
    frame_uv = frame - max_frames_organic
    if frame_uv < max_frames_organicUV:
        for idx, series in enumerate(organicUV_series):
            if frame_uv < len(series['x']):
                xdata_organicUV[idx].append(series['x'].iloc[frame_uv])
                ydata_organicUV[idx].append(series['y'].iloc[frame_uv])
                lines_organicUV[idx].set_data(xdata_organicUV[idx], ydata_organicUV[idx])
    return lines_organic + lines_organicUV

# Create the animation
ani = animation.FuncAnimation(
    fig, animate, init_func=init,
    frames=total_frames, interval=100, blit=True, repeat=True
)

# Display the animation in the notebook
HTML(ani.to_jshtml())


Organic Series:
 - BTY with 3055 data points
 - CB with 3055 data points
 - EG with 3055 data points
 - MPD with 3055 data points
 - THB with 3055 data points
 - DHB with 3055 data points

OrganicUV Series:
 - BTY UV with 3055 data points
 - CB UV with 3055 data points
 - EG UV with 3055 data points
 - MPD UV with 3055 data points
 - THB UV with 3055 data points
 - DHB UV with 3055 data points


IndexError: single positional indexer is out-of-bounds